In [18]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=8, app_name='west0_annual', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 8 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: west0_annual...


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:40375)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", 

Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:24


In [17]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


In [21]:
dates = {
    "Jan": ["2022-01-12", "2022-02-11"],
    "Feb": ["2022-02-12", "2022-03-11"],
    "March": ["2022-03-12", "2022-04-11"]
}

In [31]:
pre_dates = {
    "Oct": ["2021-10-12", "2021-11-11"],
    "Nov": ["2021-11-12", "2021-12-11"],
    "Dec": ["2021-12-12", "2022-01-10"]
}

In [24]:
for month, date in dates.items():
    print(month)
    start_date = date[0]
    end_date = date[1]
    device = "pc"
    user = load_data_mart(device, end_date, end_date, "user_master")
    membership = load_membership("pubg", device, end_date, end_date)
    user = user.withColumnRenamed("accountid", "account_id").join(membership, "account_id", "left").withColumn("membership", coalesce(col("membership"), lit("undefined")))
    user.createOrReplaceTempView("user_tmp")
    user = spark.sql("""
            SELECT *
                ,ROW_NUMBER() over(PARTITION BY account_id ORDER BY firstlogindate) as rn
            FROM user_tmp
        """) \
        .where('rn = 1').drop('rn')
    user.where(col("lastlogindate")>=start_date).groupBy("membership").agg(countDistinct("account_id")).show(truncate=False)

Jan
+----------+--------------------------+
|membership|count(DISTINCT account_id)|
+----------+--------------------------+
|plus      |87081                     |
|basic     |10798682                  |
|legacy    |7547144                   |
|undefined |10462                     |
+----------+--------------------------+

Feb
+----------+--------------------------+
|membership|count(DISTINCT account_id)|
+----------+--------------------------+
|plus      |122246                    |
|basic     |5811098                   |
|legacy    |5871091                   |
|undefined |55704                     |
+----------+--------------------------+

March
+----------+--------------------------+
|membership|count(DISTINCT account_id)|
+----------+--------------------------+
|plus      |154523                    |
|basic     |6133146                   |
|legacy    |5677139                   |
|undefined |65355                     |
+----------+--------------------------+



In [25]:
for month, date in dates.items():
    print(month)
    start_date = date[0]
    end_date = date[1]
    device = "console"
    user = load_data_mart(device, end_date, end_date, "user_master")
    membership = load_membership("pubg", device, end_date, end_date)
    user = user.withColumnRenamed("accountid", "account_id").join(membership, "account_id", "left").withColumn("membership", coalesce(col("membership"), lit("undefined")))
    user.createOrReplaceTempView("user_tmp")
    user = spark.sql("""
            SELECT *
                ,ROW_NUMBER() over(PARTITION BY account_id ORDER BY firstlogindate) as rn
            FROM user_tmp
        """) \
        .where('rn = 1').drop('rn')
    user.where(col("lastlogindate")>=start_date).groupBy("membership").agg(countDistinct("account_id")).show(truncate=False)

Jan
+----------+--------------------------+
|membership|count(DISTINCT account_id)|
+----------+--------------------------+
|plus      |22850                     |
|basic     |3685080                   |
|legacy    |1093294                   |
+----------+--------------------------+

Feb
+----------+--------------------------+
|membership|count(DISTINCT account_id)|
+----------+--------------------------+
|plus      |24508                     |
|basic     |1617874                   |
|legacy    |735940                    |
+----------+--------------------------+

March
+----------+--------------------------+
|membership|count(DISTINCT account_id)|
+----------+--------------------------+
|plus      |25500                     |
|basic     |1244286                   |
|legacy    |670282                    |
|undefined |43                        |
+----------+--------------------------+



In [8]:
for month, date in dates.items():
    print(month)
    start_date = date[0]
    end_date = date[1]
    device = "pc"
    gcoin = load_data_mart(device, start_date, end_date, "gcoin_use")
    user = load_data_mart(device, end_date, end_date, "user_master")
    user.where(col("lastlogindate") >= start_date).select(countDistinct("accountid").alias("au")).show(truncate=False)
    membership = load_membership("pubg", device, end_date, end_date)
    user = user.withColumnRenamed("accountid", "account_id").join(membership, "account_id", "left").withColumn("membership", coalesce(col("membership"), lit("undefined")))
    user.createOrReplaceTempView("user_tmp")
    user = spark.sql("""
            SELECT *
                ,ROW_NUMBER() over(PARTITION BY account_id ORDER BY firstlogindate) as rn
            FROM user_tmp
        """) \
        .where('rn = 1').drop('rn')
    
    gcoin_with_membership = gcoin.join(user.select("account_id", "membership"), "account_id", "left").withColumn("paid_account_id", when(col("paid_use") > 0, col("account_id")).otherwise(lit(None)))
    gcoin_with_membership.groupBy("membership").agg(countDistinct("paid_account_id").alias("paid_pu"), countDistinct("account_id").alias("total_pu"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).show(truncate=False)

Jan
+--------+
|au      |
+--------+
|18443369|
+--------+

+----------+-------+--------+----------+----------+
|membership|paid_pu|total_pu|paid_use  |total_use |
+----------+-------+--------+----------+----------+
|plus      |31869  |77299   |251797920 |415403670 |
|basic     |94624  |1417865 |100088820 |736517310 |
|legacy    |990562 |4339270 |1903150103|9456383260|
+----------+-------+--------+----------+----------+

Feb
+--------+
|au      |
+--------+
|11860139|
+--------+

+----------+-------+--------+----------+----------+
|membership|paid_pu|total_pu|paid_use  |total_use |
+----------+-------+--------+----------+----------+
|plus      |32565  |72460   |236472790 |349394720 |
|basic     |51402  |669181  |65439130  |343827530 |
|legacy    |444327 |1546495 |1700975646|3445541050|
+----------+-------+--------+----------+----------+

March
+--------+
|au      |
+--------+
|12030163|
+--------+

+----------+-------+--------+----------+----------+
|membership|paid_pu|total_pu|paid_us

In [22]:
for month, date in dates.items():
    print(month)
    start_date = date[0]
    end_date = date[1]
    device = "console"
    gcoin = load_data_mart(device, start_date, end_date, "gcoin_use")
    user = load_data_mart(device, end_date, end_date, "user_master")
    user.where(col("lastlogindate") >= start_date).select(countDistinct("accountid").alias("au")).show(truncate=False)
    membership = load_membership("pubg", device, end_date, end_date)
    user = user.withColumnRenamed("accountid", "account_id").join(membership, "account_id", "left").withColumn("membership", coalesce(col("membership"), lit("undefined")))
    user.createOrReplaceTempView("user_tmp")
    user = spark.sql("""
            SELECT *
                ,ROW_NUMBER() over(PARTITION BY account_id ORDER BY firstlogindate) as rn
            FROM user_tmp
        """) \
        .where('rn = 1').drop('rn')
    
    gcoin_with_membership = gcoin.join(user.select("account_id", "membership"), "account_id", "left").withColumn("paid_account_id", when(col("paid_use") > 0, col("account_id")).otherwise(lit(None)))
    gcoin_with_membership.groupBy("membership").agg(countDistinct("paid_account_id").alias("paid_pu"), countDistinct("account_id").alias("total_pu"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).show(truncate=False)

Jan
+-------+
|au     |
+-------+
|4801224|
+-------+

+----------+-------+--------+---------+---------+
|membership|paid_pu|total_pu|paid_use |total_use|
+----------+-------+--------+---------+---------+
|plus      |10311  |18388   |18206000 |44516430 |
|basic     |37450  |176915  |29026050 |92014060 |
|legacy    |93717  |350507  |125741100|587007970|
+----------+-------+--------+---------+---------+

Feb
+-------+
|au     |
+-------+
|2378322|
+-------+

+----------+-------+--------+--------+---------+
|membership|paid_pu|total_pu|paid_use|total_use|
+----------+-------+--------+--------+---------+
|plus      |6407   |11963   |13166110|26527160 |
|basic     |13194  |60448   |11622010|32459660 |
|legacy    |44035  |146701  |93545150|238819870|
+----------+-------+--------+--------+---------+

March


ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:40375)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:40375)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception

+-------+
|au     |
+-------+
|1940111|
+-------+

+----------+-------+--------+---------+---------+
|membership|paid_pu|total_pu|paid_use |total_use|
+----------+-------+--------+---------+---------+
|plus      |7297   |11863   |19394060 |31492970 |
|basic     |11033  |36645   |13506010 |25725750 |
|legacy    |54511  |134823  |131882030|255788880|
|undefined |0      |1       |0        |200      |
+----------+-------+--------+---------+---------+



## cash 매출

In [30]:
load_data_mart(device, "2022-05-05", "2022-05-05", "cash_mtx").show(truncate=False)

+----------+--------+----------------------------------------+---------------+---------+--------+-----------------------------------------------+------------+----------------+------------+----------+----------+----+----+----+--------+------+------------+-----------+-------------+---------+-------------------------+-------------------+-------------------+--------------+---------+----------------------------+-------+--------------+
|date      |platform|account_id                              |product_id     |unit_sold|sales_id|product_name                                   |product_type|category        |sub_category|event_type|event_name|tags|part|tier|currency|price_|ingame_price|gcoin_topup|survivor_pass|levels_up|start_time               |end_time           |update_time        |transaction_id|status   |time                        |is_paid|ingame_revenue|
+----------+--------+----------------------------------------+---------------+---------+--------+-----------------------------------

In [28]:
for month, date in dates.items():
    print(month)
    start_date = date[0]
    end_date = date[1]
    device = "pc"
    cash = load_data_mart(device, start_date, end_date, "cash_mtx")
    user = load_data_mart(device, end_date, end_date, "user_master")
    membership = load_membership("pubg", device, end_date, end_date)
    user = user.withColumnRenamed("accountid", "account_id").join(membership, "account_id", "left").withColumn("membership", coalesce(col("membership"), lit("undefined")))
    user.createOrReplaceTempView("user_tmp")
    user = spark.sql("""
            SELECT *
                ,ROW_NUMBER() over(PARTITION BY account_id ORDER BY firstlogindate) as rn
            FROM user_tmp
        """) \
        .where('rn = 1').drop('rn')
    
    cash_with_membership = cash.join(user.select("account_id", "membership"), "account_id", "left")
    cash_with_membership.groupBy("membership").agg(countDistinct("account_id").alias("pu"), sum("ingame_revenue").alias("ingame_revenue")).show(truncate=False)

Jan


ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:35131)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


+----------+-------+--------------+
|membership|pu     |ingame_revenue|
+----------+-------+--------------+
|plus      |87076  |4004324.79    |
|basic     |248603 |1521700.26    |
|legacy    |1402792|21247655.47   |
+----------+-------+--------------+

Feb
+----------+------+--------------+
|membership|pu    |ingame_revenue|
+----------+------+--------------+
|plus      |70982 |3337221.01    |
|basic     |100489|853141.77     |
|legacy    |491601|17524151.41   |
+----------+------+--------------+

March
+----------+------+--------------+
|membership|pu    |ingame_revenue|
+----------+------+--------------+
|plus      |87568 |3083144.81    |
|basic     |97799 |1213103.42    |
|legacy    |679936|21550320.02   |
|undefined |1     |1.99          |
+----------+------+--------------+



In [ ]:
pre_pc = {
    'month': [],
    "ingame_revenue": 
}
for month, date in pre_dates.items():
    print(month)
    start_date = date[0]
    end_date = date[1]
    device = "pc"
    cash = load_data_mart(device, start_date, end_date, "cash_mtx")
    
    cash_with_membership = cash.join(user.select("account_id", "membership"), "account_id", "left")
    cash_with_membership.groupBy("membership").agg(countDistinct("account_id").alias("pu"), sum("ingame_revenue").alias("ingame_revenue")).show(truncate=False)

In [29]:
for month, date in dates.items():
    print(month)
    start_date = date[0]
    end_date = date[1]
    device = "console"
    cash = load_data_mart(device, start_date, end_date, "cash_mtx")
    user = load_data_mart(device, end_date, end_date, "user_master")
    membership = load_membership("pubg", device, end_date, end_date)
    user = user.withColumnRenamed("accountid", "account_id").join(membership, "account_id", "left").withColumn("membership", coalesce(col("membership"), lit("undefined")))
    user.createOrReplaceTempView("user_tmp")
    user = spark.sql("""
            SELECT *
                ,ROW_NUMBER() over(PARTITION BY account_id ORDER BY firstlogindate) as rn
            FROM user_tmp
        """) \
        .where('rn = 1').drop('rn')
    
    cash_with_membership = cash.join(user.select("account_id", "membership"), "account_id", "left")
    cash_with_membership.groupBy("membership").agg(countDistinct("account_id").alias("pu"), sum("ingame_revenue").alias("ingame_revenue")).show(truncate=False)

Jan
+----------+-----+--------------+
|membership|pu   |ingame_revenue|
+----------+-----+--------------+
|plus      |21422|436042.70     |
|basic     |51366|257819.43     |
|legacy    |88314|1065450.94    |
+----------+-----+--------------+

Feb
+----------+-----+--------------+
|membership|pu   |ingame_revenue|
+----------+-----+--------------+
|plus      |11171|244009.92     |
|basic     |15096|103498.90     |
|legacy    |35981|871692.34     |
+----------+-----+--------------+

March
+----------+-----+--------------+
|membership|pu   |ingame_revenue|
+----------+-----+--------------+
|plus      |10346|274567.65     |
|basic     |11667|126720.08     |
|legacy    |45332|1258341.96    |
+----------+-----+--------------+



In [ ]:
membership = load_membership("pubg", device, target_date, target_date).select("account_id", "membership").distinct()
user = user.join(membership, "account_id", "left").withColumn("membership", coalesce(col("membership"), lit("undefined")))
user.createOrReplaceTempView("user_tmp")
user = spark.sql("""
        SELECT *
            ,ROW_NUMBER() over(PARTITION BY account_id ORDER BY firstlogindate) as rn
        FROM user_tmp
    """) \
    .where('rn = 1').drop('rn')
